# Geração de Gráficos Comparativos

Este notebook gera gráficos comparativos de performance gRPC vs RabbitMQ.

## Objetivo
Visualizar e comparar:
- RTT médio vs Número de Clientes
- RTT médio vs Tamanho de Arquivo

## Requisitos
- matplotlib
- pandas
- Arquivo `results_summary.csv` gerado pelo notebook de análise


In [1]:
import csv
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import numpy as np


## Configuração


In [2]:
# Configuração de diretórios
project_root = Path('..')
summary_file = project_root / 'results' / 'results_summary.csv'
output_dir = project_root / 'results' / 'plots'

# Cria diretório de saída
output_dir.mkdir(parents=True, exist_ok=True)

print(f"📄 Arquivo de resumo: {summary_file.absolute()}")
print(f"📁 Diretório de saída: {output_dir.absolute()}")


📄 Arquivo de resumo: /Users/ernesto/Desktop/grpc-rabbitmq-fileshare/scripts/../results/results_summary.csv
📁 Diretório de saída: /Users/ernesto/Desktop/grpc-rabbitmq-fileshare/scripts/../results/plots


## Carregamento dos Dados


In [3]:
# Carrega dados do CSV de resumo
if not summary_file.exists():
    print(f"❌ Arquivo {summary_file} não encontrado!")
    print("   Execute primeiro o notebook analyze_results.ipynb")
else:
    df = pd.read_csv(summary_file)
    print(f"✅ {len(df)} registros carregados")
    print("\n📊 Primeiras linhas:")
    display(df.head(10))


✅ 36 registros carregados

📊 Primeiras linhas:


,system,operation,file_size_kb,clients,mean_ms,stddev_ms,min_ms,max_ms
0,grpc,download,10,1,1.017,2.296,0.356,89.232
1,grpc,download,10,10,0.828,0.314,0.349,5.527
2,grpc,download,10,20,1.739,3.997,0.402,89.842
3,grpc,download,1024,1,5.177,1.357,3.784,69.097
4,grpc,download,1024,10,37.080,5.411,4.286,87.420
5,grpc,download,1024,20,80.093,19.991,5.311,250.160
6,grpc,list,0,1,0.749,0.532,0.378,30.254
7,grpc,list,0,10,0.863,0.751,0.337,21.929
8,grpc,list,0,20,0.717,0.289,0.321,3.020
9,grpc,upload,10,1,1.059,1.036,0.466,44.007


## Configuração de Estilo dos Gráficos


In [4]:
# Configuração de cores e estilo
GRPC_COLOR = '#4285F4'  # Azul Google
RABBIT_COLOR = '#EA4335'  # Vermelho Google

# Tenta usar estilo seaborn, fallback para padrão
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    try:
        plt.style.use('seaborn-darkgrid')
    except:
        plt.style.use('default')

plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 11

print("✅ Estilo configurado")


✅ Estilo configurado


## Gráficos: RTT vs Número de Clientes


In [5]:
def plot_rtt_vs_clients(df, output_dir):
    """Gera gráfico de RTT médio vs número de clientes."""
    # Agrupa por operação e tamanho de arquivo
    operations = df['operation'].unique()
    file_sizes = df['file_size_kb'].unique()
    
    plots_created = 0
    
    for operation in operations:
        for file_size_kb in file_sizes:
            # Filtra dados
            data_subset = df[
                (df['operation'] == operation) & 
                (df['file_size_kb'] == file_size_kb)
            ]
            
            if len(data_subset) == 0:
                continue
            
            fig, ax = plt.subplots(figsize=(10, 6))
            
            # Dados gRPC
            grpc_data = data_subset[data_subset['system'] == 'grpc'].sort_values('clients')
            if len(grpc_data) > 0:
                ax.plot(grpc_data['clients'], grpc_data['mean_ms'], 
                       'o-', label='gRPC', linewidth=2.5, markersize=10, 
                       color=GRPC_COLOR, markerfacecolor=GRPC_COLOR, 
                       markeredgecolor='white', markeredgewidth=1.5)
            
            # Dados RabbitMQ
            rabbit_data = data_subset[data_subset['system'] == 'rabbit'].sort_values('clients')
            if len(rabbit_data) > 0:
                ax.plot(rabbit_data['clients'], rabbit_data['mean_ms'], 
                       's-', label='RabbitMQ', linewidth=2.5, markersize=10, 
                       color=RABBIT_COLOR, markerfacecolor=RABBIT_COLOR,
                       markeredgecolor='white', markeredgewidth=1.5)
            
            ax.set_xlabel('Número de Clientes', fontweight='bold')
            ax.set_ylabel('RTT Médio (ms)', fontweight='bold')
            
            size_label = f"{int(file_size_kb)} KB" if file_size_kb > 0 else "N/A"
            title = f'RTT Médio vs Número de Clientes\n{operation.upper()} - {size_label}'
            ax.set_title(title, fontweight='bold', pad=15)
            
            ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.8)
            ax.legend(loc='best', framealpha=0.9, shadow=True)
            
            # Melhora a formatação dos eixos
            ax.set_xlim(left=0)
            ax.set_ylim(bottom=0)
            
            # Adiciona valores nos pontos
            if len(grpc_data) > 0:
                for _, row in grpc_data.iterrows():
                    ax.annotate(f"{row['mean_ms']:.1f}", 
                              (row['clients'], row['mean_ms']),
                              textcoords="offset points", xytext=(0,10), 
                              ha='center', fontsize=8, color=GRPC_COLOR)
            
            if len(rabbit_data) > 0:
                for _, row in rabbit_data.iterrows():
                    ax.annotate(f"{row['mean_ms']:.1f}", 
                              (row['clients'], row['mean_ms']),
                              textcoords="offset points", xytext=(0,10), 
                              ha='center', fontsize=8, color=RABBIT_COLOR)
            
            plt.tight_layout()
            
            # Salva o gráfico
            filename = f'rtt_vs_clients_{operation}_{int(file_size_kb)}kb.png'
            output_path = output_dir / filename
            plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
            print(f"✅ {filename}")
            plt.close()
            plots_created += 1
    
    return plots_created

print("📊 Gerando gráficos RTT vs Número de Clientes...")
plots_count = plot_rtt_vs_clients(df, output_dir)
print(f"\n✅ {plots_count} gráfico(s) criado(s)")


📊 Gerando gráficos RTT vs Número de Clientes...
✅ rtt_vs_clients_download_10kb.png
✅ rtt_vs_clients_download_1024kb.png
✅ rtt_vs_clients_download_10240kb.png
✅ rtt_vs_clients_list_0kb.png
✅ rtt_vs_clients_upload_10kb.png
✅ rtt_vs_clients_upload_1024kb.png
✅ rtt_vs_clients_upload_10240kb.png

✅ 7 gráfico(s) criado(s)


## Gráficos: RTT vs Tamanho de Arquivo


In [6]:
def plot_rtt_vs_file_size(df, output_dir):
    """Gera gráfico de RTT médio vs tamanho de arquivo."""
    # Filtra apenas operações que têm tamanho de arquivo
    df_filtered = df[df['operation'] != 'list'].copy()
    
    if len(df_filtered) == 0:
        print("⚠️  Nenhum dado disponível para gráficos de tamanho de arquivo")
        return 0
    
    operations = df_filtered['operation'].unique()
    clients_counts = df_filtered['clients'].unique()
    
    plots_created = 0
    
    for operation in operations:
        for clients in clients_counts:
            # Filtra dados
            data_subset = df_filtered[
                (df_filtered['operation'] == operation) & 
                (df_filtered['clients'] == clients)
            ]
            
            if len(data_subset) == 0:
                continue
            
            fig, ax = plt.subplots(figsize=(10, 6))
            
            # Dados gRPC
            grpc_data = data_subset[data_subset['system'] == 'grpc'].sort_values('file_size_kb')
            if len(grpc_data) > 0:
                ax.plot(grpc_data['file_size_kb'], grpc_data['mean_ms'], 
                       'o-', label='gRPC', linewidth=2.5, markersize=10, 
                       color=GRPC_COLOR, markerfacecolor=GRPC_COLOR,
                       markeredgecolor='white', markeredgewidth=1.5)
            
            # Dados RabbitMQ
            rabbit_data = data_subset[data_subset['system'] == 'rabbit'].sort_values('file_size_kb')
            if len(rabbit_data) > 0:
                ax.plot(rabbit_data['file_size_kb'], rabbit_data['mean_ms'], 
                       's-', label='RabbitMQ', linewidth=2.5, markersize=10, 
                       color=RABBIT_COLOR, markerfacecolor=RABBIT_COLOR,
                       markeredgecolor='white', markeredgewidth=1.5)
            
            ax.set_xlabel('Tamanho do Arquivo (KB)', fontweight='bold')
            ax.set_ylabel('RTT Médio (ms)', fontweight='bold')
            
            title = f'RTT Médio vs Tamanho do Arquivo\n{operation.upper()} - {int(clients)} Cliente(s)'
            ax.set_title(title, fontweight='bold', pad=15)
            
            ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.8)
            ax.legend(loc='best', framealpha=0.9, shadow=True)
            
            # Melhora a formatação dos eixos
            ax.set_xlim(left=0)
            ax.set_ylim(bottom=0)
            
            # Formata o eixo X para mostrar tamanhos de arquivo de forma legível
            ax.set_xscale('log', base=10)
            
            # Adiciona valores nos pontos
            if len(grpc_data) > 0:
                for _, row in grpc_data.iterrows():
                    ax.annotate(f"{row['mean_ms']:.1f}", 
                              (row['file_size_kb'], row['mean_ms']),
                              textcoords="offset points", xytext=(0,10), 
                              ha='center', fontsize=8, color=GRPC_COLOR)
            
            if len(rabbit_data) > 0:
                for _, row in rabbit_data.iterrows():
                    ax.annotate(f"{row['mean_ms']:.1f}", 
                              (row['file_size_kb'], row['mean_ms']),
                              textcoords="offset points", xytext=(0,10), 
                              ha='center', fontsize=8, color=RABBIT_COLOR)
            
            plt.tight_layout()
            
            # Salva o gráfico
            filename = f'rtt_vs_file_size_{operation}_{int(clients)}clients.png'
            output_path = output_dir / filename
            plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
            print(f"✅ {filename}")
            plt.close()
            plots_created += 1
    
    return plots_created

print("📊 Gerando gráficos RTT vs Tamanho de Arquivo...")
plots_count = plot_rtt_vs_file_size(df, output_dir)
print(f"\n✅ {plots_count} gráfico(s) criado(s)")


📊 Gerando gráficos RTT vs Tamanho de Arquivo...
✅ rtt_vs_file_size_download_1clients.png
✅ rtt_vs_file_size_download_10clients.png
✅ rtt_vs_file_size_download_20clients.png
✅ rtt_vs_file_size_upload_1clients.png
✅ rtt_vs_file_size_upload_10clients.png
✅ rtt_vs_file_size_upload_20clients.png

✅ 6 gráfico(s) criado(s)


## Resumo


In [7]:
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print(f"✅ Todos os gráficos salvos em: {output_dir.absolute()}")
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")

# Lista arquivos gerados
plot_files = list(output_dir.glob("*.png"))
print(f"\n📊 Total de {len(plot_files)} gráfico(s) gerado(s)")
print("\nArquivos:")
for f in sorted(plot_files):
    print(f"  - {f.name}")


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ Todos os gráficos salvos em: /Users/ernesto/Desktop/grpc-rabbitmq-fileshare/scripts/../results/plots
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📊 Total de 13 gráfico(s) gerado(s)

Arquivos:
  - rtt_vs_clients_download_10240kb.png
  - rtt_vs_clients_download_1024kb.png
  - rtt_vs_clients_download_10kb.png
  - rtt_vs_clients_list_0kb.png
  - rtt_vs_clients_upload_10240kb.png
  - rtt_vs_clients_upload_1024kb.png
  - rtt_vs_clients_upload_10kb.png
  - rtt_vs_file_size_download_10clients.png
  - rtt_vs_file_size_download_1clients.png
  - rtt_vs_file_size_download_20clients.png
  - rtt_vs_file_size_upload_10clients.png
  - rtt_vs_file_size_upload_1clients.png
  - rtt_vs_file_size_upload_20clients.png
